<a href="https://colab.research.google.com/github/SEREMM/funcs_proj_box/blob/master/06_actualized_pipeline_pred_new_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%xmode verbose
import warnings
warnings.filterwarnings("ignore")
## importar repositorio de evaluacion de libro
!wget https://raw.githubusercontent.com/hundredblocks/ml-powered-applications/master/ml_editor/model_evaluation.py
from model_evaluation import get_metrics, get_confusion_matrix_plot, get_roc_plot,\
get_calibration_plot, get_feature_importance
# Use the raw URL of the Python file
!wget https://raw.githubusercontent.com/SEREMM/funcs_proj_box/master/funciones_box_data.py
from funciones_box_data import cross_val, feat_eng, One_hot_encoder,\
Data_clusterer, Model_applied, check_fails_and_probas

Exception reporting mode: Verbose
--2023-07-25 15:55:30--  https://raw.githubusercontent.com/hundredblocks/ml-powered-applications/master/ml_editor/model_evaluation.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8593 (8.4K) [text/plain]
Saving to: ‘model_evaluation.py’

model_evaluation.py 100%[===================>]   8.39K  --.-KB/s    in 0s      

2023-07-25 15:55:30 (49.9 MB/s) - ‘model_evaluation.py’ saved [8593/8593]

--2023-07-25 15:55:31--  https://raw.githubusercontent.com/SEREMM/funcs_proj_box/master/funciones_box_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HT

#### prep data

In [ ]:
# df = pd.read_csv('box_data_duplicated_not_draw_vectorized.csv')
df = pd.read_csv('boxing_df_duplicated_not_draw_to_feat_eng.csv')

In [ ]:
y_df = df[['result','endtype','endround']]
y_df.drop(columns='endround', inplace=True)
x_df = df.drop(columns=['result','endtype','endround'])

In [ ]:
y_df = y_df.result.replace({'loss':-1,'win':1})

In [ ]:
from sklearn.model_selection import train_test_split
# train data
xtrain,xval,ytrain,yval = train_test_split(x_df,y_df, test_size=.35,
                                           random_state=40, stratify=y_df)
# val test data
xval,xtest,yval,ytest = train_test_split(xval,yval, test_size=.15,
                                           random_state=40, stratify=yval)
# xl data
xl_train,xl_test,yl_train,yl_test = train_test_split(x_df,y_df, test_size=.30,
                                           random_state=40, stratify=y_df)

#### 1st pipeline : ranfor

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vecto_reg = CountVectorizer(binary=True)
regiones = ['north america','usa','mexico','caribe','latin america',
            'south america','argentina',
            'uk','west europe','north europe','east europe',
            'north africa','south africa',
            'west asia','ukraine','russia','middle asia','east asia',
            'oceania','pacific']
vecto_reg.fit(regiones)

vecto_style = CountVectorizer(binary=True)
estilos = ['puncher','aggressive','brawler','pressure',
          'technical','slugger','swarmer','slick','counter','defensive']
vecto_style.fit(estilos)

vecto_pos = CountVectorizer(binary=True)
postura = ['orthodox','southpaw']
vecto_pos.fit(postura)

CountVectorizer(binary=True)

In [ ]:
# feature engineering
xtrain_0 = feat_eng(xtrain)

# vectorizacion
encoder = One_hot_encoder(vecto_reg, vecto_pos, vecto_style)
xtrain_1 = encoder.transform(X=xtrain_0)

from sklearn.cluster import KMeans

clus = KMeans(n_clusters=5, random_state=10)
clus.fit(xtrain_1)
cluster = Data_clusterer(clus)
xtrain_2 = cluster.transform(xtrain_1)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
xtrain_3 = scaler.fit_transform(xtrain_2)

from sklearn.ensemble import RandomForestClassifier

ranfor = RandomForestClassifier()

ranfor.fit(xtrain_3, ytrain)

RandomForestClassifier()

In [ ]:
model = Model_applied(ranfor)

x_0 = feat_eng(xtest)
x_1 = encoder.transform(x_0)
x_clus = cluster.transform(x_1)
x_scal = scaler.transform(x_clus)
y_pred = model.transform(x_scal)

xtest_0 = feat_eng(xtest)
xtest_1 = encoder.transform(xtest_0)
xtest_clus = cluster.transform(xtest_1)
xtest_scal = scaler.transform(xtest_clus)
ytest_pred = model.transform(xtest_scal)

In [ ]:
get_metrics(yl_test, y_pred.boxer1_pred)

ValueError: ignored

In [ ]:
get_metrics(ytest, ytest_pred.boxer1_pred)

In [ ]:
feature_names = xtrain_2.columns

k = 40
print("Top %s importances:\n" % k)
print('\n'.join(["%s: %.2g" % (tup[0], tup[1]) for tup in get_feature_importance(ranfor, feature_names)[:k]]))

print("\nBottom %s importances:\n" % k)
print('\n'.join(["%s: %.2g" % (tup[0], tup[1]) for tup in get_feature_importance(ranfor, feature_names)[-k:]]))

In [ ]:
get_confusion_matrix_plot(y_pred.boxer1_pred, ytest,
                          figsize=(5, 3), classes=['-1 : loss','1 : win'])
plt.show()

In [ ]:
proba_win = y_pred.prob_win

In [ ]:
get_roc_plot(proba_win, ytest, figsize=(5,3))
plt.show()

In [ ]:
get_calibration_plot(proba_win, ytest, figsize=(5, 5))

In [ ]:
dfx = check_fails_and_probas(x_clus, ytest, y_pred.boxer1_pred, y_pred.prob_loss, y_pred.prob_win)

#### cross val hyperparams

In [ ]:
# search for best hyperparameters
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_dist = {
    'n_estimators': randint(50, 200),
    'max_depth': [None, 5, 10],
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 4)
}

# Perform randomized search to find the best hyperparameters
random_search = RandomizedSearchCV(estimator=ranfor, param_distributions=param_dist, n_iter=10, cv=5)
random_search.fit(xtrain_3, y_dfnovmas)

# Print the best hyperparameters found
print("Best hyperparameters:")
print(random_search.best_params_)

# Evaluate the best model on the test set
best_model = random_search.best_estimator_
accuracy = best_model.score(xl_test, yl_test)
print("Accuracy on test set:", accuracy)

Best hyperparameters:
{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 117}


ValueError: ignored

In [ ]:
# model with hyperparameters
ranfor_hype = RandomForestClassifier(max_depth=None,
                                min_samples_leaf=1,
                                min_samples_split=2,
                                n_estimators=180)

# ranfor hype
print('validación cruzada ranfor sobre data xl ranfor hype\n')
cross_val(ranfor_hype, xl_train_scaled, xl_test_scaled,
          yl_train.result, yl_test.result)

#### new preds

In [ ]:
dfnovmas = df[df.y_f > 1994]
y_dfnovmas = dfnovmas[['result','endtype','endround']]
y_dfnovmas.drop(columns='endround', inplace=True)
x_dfnovmas = dfnovmas.drop(columns=['result','endtype','endround'])
y_dfnovmas = y_dfnovmas.result.replace({'loss':-1,'win':1})

In [ ]:
dfnovmas.shape

(4426, 31)

In [ ]:
# feature engineering
xtrain_0 = feat_eng(x_dfnovmas)

# vectorizacion
encoder = One_hot_encoder(vecto_reg, vecto_pos, vecto_style)
xtrain_1 = encoder.transform(X=xtrain_0)

from sklearn.cluster import KMeans

clus = KMeans(n_clusters=5, random_state=10)
clus.fit(xtrain_1)
cluster = Data_clusterer(clus)
xtrain_2 = cluster.transform(xtrain_1)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
xtrain_3 = scaler.fit_transform(xtrain_2)

from sklearn.ensemble import RandomForestClassifier

ranfor = RandomForestClassifier(max_depth=None, min_samples_leaf=1, min_samples_split=3, n_estimators=117)

ranfor.fit(xtrain_3, y_dfnovmas)

RandomForestClassifier(min_samples_split=3, n_estimators=117)

In [ ]:
# modelo fit in xl, aun no toda la data
model = ranfor
# df = pd.read_csv('/content/box_data_duplicated_not_draw_feat_eng_vectorized.csv').iloc[:,3:]
newdf = pd.read_csv('/content/base_datos_peleas_new - Hoja 1.csv')
pelea = newdf.iloc[:,0:5]
test_df = newdf.iloc[:,5:]

In [ ]:
# feature engineering
test_df = feat_eng(test_df)

# vectorizacion
test_df = encoder.transform(test_df)

# cluster scal and pred
test_df_clus = cluster.transform(test_df)
test_df_scal = scaler.transform(test_df_clus)
test_df_pred = Model_applied(model).transform(test_df_scal)

In [ ]:
temp = test_df_clus[['birth_b1','height_b1','reach_b1','birth_b2',
       'height_b2','reach_b2','b1_w','b1_wk','b1_d','b1_l','b1_lk','b2_w','b2_wk','b2_d','b2_l',
       'b2_lk','b1_bet','b2_bet','cluster']]

In [ ]:
pd.concat([pelea, test_df_pred, temp['cluster']], axis=1)

,result,end_round,end_type,boxer1,boxer2,boxer1_pred,prob_win,prob_loss,cluster
0,win,12.0,desc,canelo,jhon ryder,1,0.556980,0.443020,2
1,win,7.0,ko,gtd,ryan garcia,1,0.695869,0.304131,1
2,win,12.0,desc,devin haney,lomachenko,1,0.655271,0.344729,1
3,win,12.0,desc,josh kelly,gabriel corzo,1,0.629630,0.370370,1
4,win,12.0,desc,andy cruz,juan carlos burgos,1,0.517094,0.482906,1
5,win,12.0,desc,george kambosos,maxi hughes,1,0.658832,0.341168,1
6,loss,8.0,ko,stephen fulton,naoya inoue,1,0.507835,0.492165,1
7,win,5.0,ko,robeisy ramirez,satoshi shimizu,1,0.501994,0.498006,1
8,NaN,NaN,NaN,jake paul,nate diaz,1,0.563818,0.436182,1
